# **道路跟随 - 采集数据（多目标）**
使用 ``regression`` 预测出前进的目标地点，实现巡线

1. 将JetRacer放置在路径上的不同位置（偏离中心，不同角度等）
2. 显示来自机器人的实时摄像头
3. 使用游戏手柄控制器，在图像上放置一个“绿点”，该绿点对应于我们希望机器人行进的目标方向。
4. 存储该绿点的X，Y值以及机器人摄像机的图像（**x, y值为target在图像上以左上角为原点的相对位置**）

然后，在训练笔记本中，我们将训练神经网络来预测标签的X，Y值。 在现场演示中，我们将使用预测的X，Y值以计算近似的转向值。

>数据采集建议：
>1. 想象一下机器人应遵循的路径，按照预设较好的路线进行数据采集。
>2. 直道时，可将目标点设置的相对远一些。
>3. 直道转弯道时，时刻观察JetRacer所处位置和摄像头采集视频，准确估计入弯时机
>4. 在弯道时，每移动一小步就进行小距离的标点。

### **导入依赖**

In [2]:
#Add search path
import sys
sys.path.append('../')
# IPython Libraries for display and widgets
import ipywidgets
import traitlets
import ipywidgets.widgets as widgets
from jupyter_clickable_image_widget import ClickableImageWidget
from IPython.display import display


# Python basic pakcages for image annotation
from uuid import uuid1 #命名唯一ID
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

# Camera for Jetracer
from camera import Camera, bgr8_to_jpeg

### **开启摄像头**

In [6]:
image_width = 224#
image_height = 224#
display_width = 224
display_height = 224
camera_fps = 30

In [7]:
camera = Camera(width=image_width, height=image_height, fps=camera_fps)#启动摄像头

### **定义目标采集器**
定义采集器类

In [11]:
class CreateTargetWidget():
    def __init__(self, camera, display_width, display_height, name):
        #定义变量
        self.camera = camera
        self.width = display_width
        self.height = display_height
        self.name = name
        #定义控件
        self.target_widget = ClickableImageWidget(width=self.width, height=self.height)
        self.x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, value=0.0)
        self.y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, value=0.0)
        time.sleep(1)
        #设置链接
        traitlets.dlink((self.camera, 'value'), (self.target_widget, 'value'), transform=self.display_xy)
        self.target_widget.on_msg(self.change_xy)
        #显示
        #display(self.target_widget, self.x_slider, self.y_slider)
        
        
    def display_xy(self, camera_image):
        #数据获取
        image = np.copy(camera_image)
        image = cv2.resize(image, (self.width, self.height))
        #画出辅助线
        image = cv2.line(image, (int(self.width/2), 0), (int(self.width/2), self.height-1), (200, 200, 200),2)
        image = cv2.line(image, (0, int(self.height/2)), (self.width-1, int(self.height/2)), (200, 200, 200),2)
        #画出摄像头中心，目标中心，两点连线
        x = self.x_slider.value
        y = self.y_slider.value
        x = int((x+1) * self.width / 2)#使（0，0）处于图像正中间
        y = int((y+1) * self.height / 2)
        image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
        image = cv2.circle(image, (int(self.width/2), self.height-1), 8, (0, 0,255), 3)
        image = cv2.line(image, (x,y), (int(self.width/2), self.height-1), (255,0,0), 3)
        jpeg_image = bgr8_to_jpeg(image)
        return jpeg_image
    
    def change_xy(self, _, content, msg):
        if content['event'] == 'click':
            data = content['eventData']
            x = round(data['offsetX']/(self.width/2) - 1, 2)
            #y = round((data['offsetY'] - 112)*2/224, 2)
            y = round(data['offsetY']/(self.height/2) - 1, 2)
            #text.value += 'offset: (%d, %d) slider: (%.2f, %.2f)'%(data['offsetX'],data['offsetY'], x, y)
            #text.value += '(%d, %d)\n'%(data['offsetX'],data['offsetY'])
            self.x_slider.value = x
            self.y_slider.value = y
    def display(self):
        return ipywidgets.VBox([
            ipywidgets.Label('-'*20+self.name+'-'*20),
            self.target_widget, 
            self.x_slider, 
            self.y_slider
        ])
    
    def value(self):
        return [self.x_slider.value, self.y_slider.value]
           

实例化目标采集器

>可通过修改``target_names``生成多个目标采集器

In [12]:
target_names = ['main(left)', 'secondary(right)']

target_widgets = []
for name in target_names:
    target_widgets.append(CreateTargetWidget(camera, display_width, display_height, name))

显示可交互控件

In [13]:
display(ipywidgets.HBox([
    target_widgets[0].display(),
    target_widgets[1].display()
]))

### **保存数据**
定义保存功能按钮``save_widget``及计数按钮``count_widget``

>可通过修改``DATASET_DIR``修改数据保存地址

In [16]:
DATASET_DIR = 'data/dataset_xy_1'

# 创建数据集文件夹
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

# 创建数据集计数控件
count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))
# 创建保存按钮
save_widget = widgets.Button(description='SAVE',button_style='warning')

def xy_uuid(target_widgets):
    save_name = 'xy_'
    for target in target_widgets:
        data = target.value()
        save_name +='%03d_%03d_'%(data[0] * 50 + 50, data[1] * 50 + 50)
    save_name += '%s'%(uuid1())
    #return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())
    return save_name

#将图片保存在本地，同时转换为数据集命名格式
def save_snapshot():
    uuid = xy_uuid(target_widgets)
    image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
    with open(image_path, 'wb') as f:
        image = np.copy(camera.value)
        #image = correctImage(image, coefficient_group)
        #image = cv2.resize(image, (save_width, save_height))
        image = bgr8_to_jpeg(image)
        f.write(image)
    count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

def widget_save(change):
    save_snapshot()

#将保存按钮与保存功能关联
save_widget.on_click(widget_save)

Directories not created becasue they already exist


### **JetRacer 运动采集（可选项）**
使用触控/手柄控制 **Jetracer** 行动的同时，进行数据采集
>默认通过手动不断摆放 Jetracer 到不同的位置采集数据

In [23]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar() #实例化Jetracer对象

In [33]:
car.throttle_gain = 0.15 #限制Jetracer最高速度为0.2
car.steering_offset= -0.037  #设置前轮默认偏移量
#car.steering_gain=-0.8

### 添加手柄控制

In [10]:
controller = widgets.Controller(index=0)
display(controller)

Controller()

In [11]:
left_link = traitlets.dlink((controller.axes[0], 'value'), (car, 'steering'), transform=lambda x: x)
right_link = traitlets.dlink((controller.axes[5], 'value'), (car, 'throttle'), transform=lambda x: -x)

### 显示实时浏览摄像头及交互控件
**运动控制：**

使用左摇杆``横轴``控制 Jetracer 横向运动

使用右摇杆``纵轴``控制 Jetracer 纵向运动

**绿色圈控制：**
使用``slider``控制x，y坐标或**鼠标点击显示框**

**保存**
点击``save``按钮保存<br>

In [18]:
display(
    widgets.HBox([target.display() for target in target_widgets]),
    count_widget,
    save_widget
)

IntText(value=823, description='count')

Button(button_style='warning', description='SAVE', style=ButtonStyle())

In [15]:
camera.stop()